In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

# Misal kita punya DataFrame dengan kolom 'user_id', 'item_id', dan 'rating'
# Sesuaikan dataset Anda dengan kolom yang sesuai

df = pd.read_csv('rating_final.csv')

from sklearn.preprocessing import LabelEncoder

# Contoh pemetaan untuk user_id
user_encoder = LabelEncoder()
df['userID'] = user_encoder.fit_transform(df['userID'])

# Pemetaan untuk item_id
item_encoder = LabelEncoder()
df['placeID'] = item_encoder.fit_transform(df['placeID'])

# Split data menjadi data latih dan validasi
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)




# Jumlah pengguna dan item
num_users = df['userID'].nunique()
num_items = df['placeID'].nunique()

# Menentukan ukuran embedding
embedding_size = 50

# Input layer
user_input = Input(shape=(1,), name='user_input')
item_input = Input(shape=(1,), name='item_input')

# Embedding layer for GCF
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_size, input_length=1)(user_input)
item_embedding = Embedding(input_dim=num_items, output_dim=embedding_size, input_length=1)(item_input)

# Flatten embeddings
user_flat = Flatten()(user_embedding)
item_flat = Flatten()(item_embedding)

# Concatenate embeddings for GCF
concatenated_gcf = Concatenate()([user_flat, item_flat])

# Multi-Layer Perceptron (MLP)
mlp_input = Concatenate()([user_embedding, item_embedding])
mlp_flatten = Flatten()(mlp_input)
mlp_fc1 = Dense(64, activation='relu')(mlp_flatten)
mlp_output = Dense(1)(mlp_fc1)

# Combine GCF and MLP outputs
combined = Concatenate()([concatenated_gcf, mlp_output])

# Fully connected layers for final prediction
final_fc1 = Dense(64, activation='relu')(combined)
output = Dense(1)(final_fc1)

# Model
model = Model(inputs=[user_input, item_input], outputs=output)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss=MeanSquaredError())

# Training model
model.fit([train_data['userID'], train_data['placeID']], train_data['rating'], epochs=100, batch_size=2, validation_data=([val_data['userID'], val_data['placeID']], val_data['rating']))

Epoch 1/100
464/464 [==============================] - 3s 3ms/step - loss: 0.6782 - val_loss: 0.4019
Epoch 2/100
464/464 [==============================] - 1s 2ms/step - loss: 0.3551 - val_loss: 0.4149
Epoch 3/100
464/464 [==============================] - 1s 3ms/step - loss: 0.3144 - val_loss: 0.4218
Epoch 4/100
464/464 [==============================] - 1s 2ms/step - loss: 0.2908 - val_loss: 0.4269
Epoch 5/100
464/464 [==============================] - 1s 2ms/step - loss: 0.2796 - val_loss: 0.4847
Epoch 6/100
464/464 [==============================] - 1s 2ms/step - loss: 0.2506 - val_loss: 0.4766
Epoch 7/100
464/464 [==============================] - 1s 2ms/step - loss: 0.2232 - val_loss: 0.5048
Epoch 8/100
464/464 [==============================] - 1s 2ms/step - loss: 0.1895 - val_loss: 0.5397
Epoch 9/100
464/464 [==============================] - 1s 3ms/step - loss: 0.1553 - val_loss: 0.5419
Epoch 10/100
464/464 [==============================] - 2s 4ms/step - loss: 0.1231 - val_lo